In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import gc
gc.collect()

E:\Anaconda3\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0

# <font color=red>=====11/6 18：00之后A和B榜数据加载======<font>

In [3]:
train_data=pd.read_table('../data/train.txt',names= ['prefix','query','title','tag','label'],
                         header= None, encoding='utf-8').astype(str) 


val_data=pd.read_table('../data/vali.txt',names = ['prefix','query','title','tag','label'],
                       header = None, encoding='utf-8').astype(str)

testa=pd.read_table('../data/testa.txt',names=['prefix','query','title','tag'],
                   header=None,encoding='utf-8').astype(str)

testb=pd.read_table('../data/testb.txt',names=['prefix','query','title','tag'],
                   header=None,encoding='utf-8').astype(str)

print(testa.label.value_counts(),testb.label.value_counts())

In [ ]:
testa['label']=-1
testb['label']=-2
test_data=pd.concat([testa,testb],axis=0)
testdf=test_data

train_data = train_data[train_data['label'] != '音乐' ]
traindf= pd.concat([train_data,val_data])

# ==========开始数据处理===========

In [4]:
traindf['label'] = traindf['label'].apply(lambda x: int(x))
testdf['label'] = testdf['label'].apply(lambda x: int(x))

data=pd.concat([traindf,testdf],axis=0)
print(traindf.shape,testdf.shape,data.shape)
print(traindf.label.value_counts())

(2049998, 5) (50000, 5) (2099998, 5)
0    1287218
1     762780
Name: label, dtype: int64


In [5]:
data['query_len']=data['query'].apply(lambda x:len(x.replace('{','').replace('}','').split(',')))
data['prefix_len']=data['prefix'].apply(lambda x:len(str(x)))
data['title_len']=data['title'].apply(lambda x:len(str(x))) 

le=LabelEncoder()
le_cols=['prefix','title','query','tag']
for feature in le_cols:
    data[feature]=le.fit_transform(data[feature].apply(str))

In [6]:
traindf=data[data['label']> -1]
testdf=data[data['label'] < = -1]
print(traindf.shape,testdf.shape)
    
col=['prefix','query','title','tag','prefix_len','query_len','title_len']
for item in col:
    temp=traindf.groupby(item,as_index=False)['label'].agg({item+'_click':'sum',item+'_count':'count'})
    temp[item+'_ctr']=(temp[item+'_click']/temp[item+'_count'])
    traindf = pd.merge(traindf, temp, on=item, how='left')
    testdf=pd.merge(testdf,temp,on=item,how='left')
    
for i in range(len(col)):
    for j in range(i+1,len(col)):
        item_g=[col[i],col[j]]
        temp=traindf.groupby(item_g,as_index=False)['label'].agg({'_'.join(item_g)+'_click':'sum','_'.join(item_g)+'_count':'count'})
        temp['_'.join(item_g)+'_ctr']=temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'_count']+3)
        traindf=pd.merge(traindf,temp,on=item_g,how='left')
        testdf=pd.merge(testdf,temp,on=item_g,how='left')

(2049998, 8) (50000, 8)


#<font color=red> =====check下testdf中有多少NaN值，如果太多的话，则会对下面的训练造成的噪声比较大======<font>

In [ ]:
# nan_len=set(testdf.columns[np.where(pd.isnull(testdf))[1]])

# print(len(nan_len))

#<font color=red> ======check结果，10%的话可以接受，否则要增加更多的辨别特征============<font>

In [7]:
data=pd.concat([traindf,testdf],axis=0)

# ==============ctr feature之后，有一些数据是在test中有，train中没有的，所以会产生nan============

In [8]:
#找到testdf中含有NaN值的列名！！！
nan_col=set(testdf.columns[np.where(pd.isnull(testdf))[1]])
print(len(nan_col))

{'query_title_len_ctr', 'title_title_len_ctr', 'title_prefix_len_click', 'query_len_click', 'query_tag_ctr', 'prefix_query_count', 'prefix_title_count', 'prefix_len_query_len_click', 'prefix_len_query_len_count', 'tag_query_len_count', 'query_query_len_ctr', 'prefix_prefix_len_ctr', 'prefix_ctr', 'query_prefix_len_count', 'query_len_title_len_count', 'query_title_count', 'query_prefix_len_click', 'query_prefix_len_ctr', 'title_prefix_len_ctr', 'prefix_title_len_count', 'title_click', 'query_count', 'title_query_len_ctr', 'title_tag_click', 'query_click', 'prefix_len_query_len_ctr', 'query_len_title_len_click', 'prefix_click', 'prefix_query_len_click', 'title_ctr', 'tag_title_len_click', 'prefix_len_title_len_click', 'tag_query_len_click', 'prefix_len_title_len_ctr', 'title_tag_count', 'prefix_title_click', 'title_tag_ctr', 'prefix_tag_click', 'prefix_count', 'tag_title_len_count', 'prefix_title_len_click', 'query_len_count', 'prefix_tag_count', 'query_tag_count', 'query_query_len_count

In [12]:
if 'label' in nan_col:
    print('yes')
else:
    print('no')

no


In [15]:
nan_list=[]
for fea in nan_col:
    nan_list.append(fea)
print(len(nan_list))

for feature in nan_list:
    data[feature]=data[feature].fillna('-1')
    data[feature]=le.fit_transform(data[feature].apply(str))

72


In [16]:
print(pd.isnull(data).sum())

prefix                        0
query                         0
title                         0
tag                           0
label                         0
query_len                     0
prefix_len                    0
title_len                     0
prefix_click                  0
prefix_count                  0
prefix_ctr                    0
query_click                   0
query_count                   0
query_ctr                     0
title_click                   0
title_count                   0
title_ctr                     0
tag_click                     0
tag_count                     0
tag_ctr                       0
prefix_len_click              0
prefix_len_count              0
prefix_len_ctr                0
query_len_click               0
query_len_count               0
query_len_ctr                 0
title_len_click               0
title_len_count               0
title_len_ctr                 0
prefix_query_click            0
                             ..
title_ta

# <font color=red>===========基本数据处理完成，下面开始分成train and test===========<font>

In [17]:
traindf=data[:traindf.shape[0]]
testdf=data[traindf.shape[0]:]
print(traindf.shape,testdf.shape)
print(traindf.label.value_counts())
print(testdf.label.value_counts())

(2049998, 92) (50000, 92)


In [18]:
sub_train=pd.DataFrame()
sub_train['org_label']=traindf['label']

In [20]:
########################第1种验证方式：即全部的orginal val data##################################
valdf1=traindf[(train_data.shape[0]):]
print(valdf1.shape)
print(valdf1.label.value_counts())

# #用于第二层stack的验证方式一
# val_train1=pd.DataFrame()
# val_train1['label']=valdf1['label']

y1=valdf1.pop('label').values
X1=valdf1.as_matrix()

(50000, 92)
0    31415
1    18585
Name: label, dtype: int64


In [22]:
########################第2种验证方式：即train_set的随机的15%############################################
valdf2=traindf[:train_data.shape[0]]
print(valdf2.shape)
print(valdf2.label.value_counts())

y2=valdf2.pop('label').values
X2=valdf2.as_matrix()
X_train,X_test,y_train,y_test=train_test_split(X2,y2,test_size=0.15,random_state=2018)

# #用于第二层stack的验证2
# val_y2=y_test.reshape(-1,1)  

# ==============基本stacking思路===========

In [21]:
testdf=testdf.drop(['label'],axis=1)
y=traindf.pop('label').values
X=traindf.as_matrix()
testX=testdf.as_matrix()

In [23]:
# lgb可以得到:

# 10_train---sub_train

# 10_val==one_whole_train(stack)---stacklgb

# 10_test---sub_test

# 第1个成绩：stack_lr_fit(10_val,label)
#             stack_lr_predict(10_test)

# 第2个成绩：stack_lr_fit(mean(mean_10_train,10_val),label)
#            stack_lr_predict(10_test)

# 第3个成绩：stack_lgb_fit(10_train,label)
#            stack_lgb_predict(10_test)

# 第4个成绩：averaging(1,2,3)----递交！

# ==========开始lgb训练=============

In [24]:
lgb= LGBMClassifier(boosting_type='gbdt', num_leaves=60, max_depth=-1, 
                         learning_rate=0.05, n_estimators=2000,objective='binary', 
                         min_split_gain=0,min_child_weight=5, min_child_samples=10, 
                         subsample=0.8, subsample_freq=1,colsample_bytree=0.8, 
                         reg_alpha=3, reg_lambda=5, random_state=1, n_jobs=-1)

N=10

ntrain=len(traindf)
ntest=len(testdf)

sub_test=pd.DataFrame()
stack_train=np.zeros((ntrain,))
stack_test=np.zeros((ntest,))
stack_test_i=np.empty((N,ntest))

ft=0
fv=0
rd=0
va=0

In [25]:
skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)
for i,(train_index,test_index) in enumerate(skf.split(X,y)):
    print("Fold=%s"%(i))
    model=lgb.fit(X[train_index],y[train_index],
                   eval_names=['train','valid'],eval_metric='logloss',
                    eval_set=[(X[train_index],y[train_index]),(X[test_index],y[test_index])],
                    early_stopping_rounds=100,verbose=100,)
    
    print("best logloss on train_set:",(model.best_score_['train']['binary_logloss']))
    print("best logloss on val_set:",(model.best_score_['valid']['binary_logloss']))
    
    ###################第1种验证方式：val data##############################################
    va_pred=model.predict_proba(X1)[:,1]
    va_f=f1_score(y1,np.where(va_pred>0.37,1,0))
    print("f1 score on orginal val data:%.5f"%(va_f))
    va+=(va_f)
    ########################################################################################

    ###################第2种验证方式：train_set的随机的15%######################################
    rand_pred=model.predict_proba(X_test)[:,1]
    rand_f=f1_score(y_test,np.where(rand_pred>0.37,1,0))
    print("f1 score of all traindf's random 15percent:%.5f"%(rand_f))
    rd+=(rand_f)
    ########################################################################################
    
    train_all=model.predict_proba(X,num_iteration=model.best_iteration_)[:,1]
    sub_train['ans_%s'%str(i)]=train_all
    
    train_proba=model.predict_proba(X[train_index],num_iteration=model.best_iteration_)[:,1]
    ftrain=f1_score(y[train_index],(np.where(train_proba>0.37,1,0)))
    print("f1_score on train_set:%.5f"%ftrain)
    ft+=ftrain
    
    val_proba=model.predict_proba(X[test_index],num_iteration=model.best_iteration_)[:,1]
    fval=f1_score(y[test_index],(np.where(val_proba>0.37,1,0)))
    print("f1_score on val_set:%.5f"%fval)
    fv+=fval
    
    #将每一次的Val进行保存，以备后续stacking直接使用，由于是ndarrya类型，故可以直接fit(ndarrayX,ndarrayY)
    stack_train[test_index]=val_proba
        
    test_proba=model.predict_proba(testX,num_iteration=model.best_iteration_)[:,1]
    sub_test['ans_%s'%str(i)]=test_proba
    stack_test_i[i,:]=test_proba
stack_test[:]=stack_test_i.mean(axis=0)

print("mean f1 score on train set:%.5f"%(ft/N))
print("mean f1 score on val set:%.5f"%(fv/N))
print("验证方式1：mean f1 score on orginal val data:%.5f"%(va/N))
print("验证方式2：mean f1 score on random 15percent:%.5f"%(rd/N))
print(sub_train.shape,sub_test.shape) 
#sub_train.shape应该是（2049998，11）,sub_test应该是（50000，10）

Fold=0
Training until validation scores don't improve for 100 rounds.
[100]	train's binary_logloss: 0.325127	valid's binary_logloss: 0.325519
[200]	train's binary_logloss: 0.322455	valid's binary_logloss: 0.3236
[300]	train's binary_logloss: 0.321828	valid's binary_logloss: 0.323825
Early stopping, best iteration is:
[222]	train's binary_logloss: 0.322263	valid's binary_logloss: 0.323582
best logloss on train_set: 0.3222633652612274
best logloss on val_set: 0.32358150366951666
f1 score on orginal val data:0.82998
f1 score of all traindf's random 15percent:0.80139
f1_score on train_set:0.80080
f1_score on val_set:0.79958
Fold=1
Training until validation scores don't improve for 100 rounds.
[100]	train's binary_logloss: 0.325193	valid's binary_logloss: 0.324775
[200]	train's binary_logloss: 0.322575	valid's binary_logloss: 0.32282
[300]	train's binary_logloss: 0.321931	valid's binary_logloss: 0.32297
Early stopping, best iteration is:
[215]	train's binary_logloss: 0.322425	valid's binary

# ============stack_by_lr===============

In [26]:
sub_train.to_csv('../data/stack/sub_train.csv')
sub_test.to_csv('../data/stack/sub_test.csv')

# ==========lr with gridsearch======

In [30]:
# from sklearn.model_selection import GridSearchCV

# sub_lr_cv=pd.DataFrame()

# x=stack_train.reshape(-1,1)
# Y=sub_train['org_label'].values
# testtx=stack_test.reshape(-1,1)

# xtrain,xtest,ytrain,ytest=train_test_split(x,Y,test_size=0.2,random_state=2018)

In [36]:
# param_grid={ 'solver':['saga','liblinear'],
#             'tol':[0.0001,0.0005,0.002],
#             'C':[0.05,0.08,1.2]
#            }

# clf=LogisticRegression()
# lr_cv=GridSearchCV(clf,param_grid,cv=10)
# lr_cv.fit(xtrain,ytrain)

# lr_pred=lr_cv.predict_proba(xtest)[:,1]
# print(lr_cv.best_params_)
# print(lr_cv.best_score_)
# print(f1_score(ytest,np.where(lr_pred>0.37,1,0))

KeyboardInterrupt: 

In [35]:
# best_clf=lr_cv.best_estimator_
# best_clf.fit(xtrain,ytrain)
# lr_tr=best_clf.predict_proba(xtrain)[:,1]
# lr_val=best_clf.predict_proba(xtest)[:,1]
# print(f1_score(ytrain,np.where(lr_tr>0.37,1,0))
# print(f1_score(ytest,np.where(lr_val>0.37,1,0))
# lr_test=best_clf.predict_proba(testtx)[:,1]
# sub_lr_cv['label']=lr_test
# sub_lr_cv['label']=sub_lr_cv['label'].apply(lambda x:np.where(x>0.37,1,0))
# print(sub_lr_cv.label.value_counts())
# sub_lr_cv['label'].to_csv('../data/BaselineV1_8_stackcv.csv',index=False,header=False)

# ========通过上面找到了最好的参数=======

In [37]:
gc.collect()

1155

In [38]:
lr=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.08, random_state=1,
                            solver='liblinear', max_iter=1000,verbose=50)

sub_lr=pd.DataFrame()

x=stack_train.reshape(-1,1)
Y=sub_train['org_label'].values
testx=stack_test.reshape(-1,1)

for i,(train_index,test_index) in enumerate(skf.split(x,Y)):
    print("Fold=%s"%i)
    lr.fit(x[train_index],Y[train_index])
    
    train_pred=lr.predict_proba(x[train_index])[:,1]
    trainf=f1_score(Y[train_index],(np.where(train_pred>0.37,1,0)))
    print("f1 in train set:",trainf)
    
    val_pred=lr.predict_proba(x[test_index])[:,1]
    valf=f1_score(Y[test_index],(np.where(val_pred>0.37,1,0)))
    print("f1 in val set:",valf)

    test_pred=lr.predict_proba(testx)[:,1]
    sub_lr['ans_%s'%str(i)]=test_pred
print(sub_lr.shape)

Fold=0
[LibLinear]f1 in train set: 0.800474420180798
f1 in val set: 0.8005460978361194
Fold=1
[LibLinear]f1 in train set: 0.8003172562178587
f1 in val set: 0.8019621906859983
Fold=2
[LibLinear]f1 in train set: 0.8005553123097787
f1 in val set: 0.8000358519315228
Fold=3
[LibLinear]f1 in train set: 0.8005851741640623
f1 in val set: 0.799571660329206
Fold=4
[LibLinear]f1 in train set: 0.8005522238115548
f1 in val set: 0.7998463262365935
Fold=5
[LibLinear]f1 in train set: 0.8006233257375297
f1 in val set: 0.7992483035519581
Fold=6
[LibLinear]f1 in train set: 0.8002748113341875
f1 in val set: 0.8023499413795975
Fold=7
[LibLinear]f1 in train set: 0.800581505863826
f1 in val set: 0.7996489789643598
Fold=8
[LibLinear]f1 in train set: 0.8004852582170321
f1 in val set: 0.8006453512939037
Fold=9
[LibLinear]f1 in train set: 0.8004322412692176
f1 in val set: 0.8009648323732054
(50000, 10)


In [39]:
sub_lr['label']=0
for i in range(N):
    sub_lr['label']+=sub_lr['ans_%s'%str(i)]
sub_lr['label']=sub_lr['label']/N
sub_lr['label']=sub_lr['label'].apply(lambda x:np.where(x>0.37,1,0))

print(sub_lr.label.value_counts())
print(sub_lr.shape)

# print(     /250000)

0    33198
1    16802
Name: label, dtype: int64
(50000, 11)
      ans_0     ans_1     ans_2     ans_3     ans_4     ans_5     ans_6  \
0  0.046847  0.046916  0.046900  0.046810  0.046830  0.046803  0.046927   
1  0.596051  0.595954  0.596242  0.596111  0.596086  0.596009  0.595897   
2  0.654571  0.654447  0.654743  0.654644  0.654612  0.654544  0.654385   
3  0.043151  0.043217  0.043201  0.043116  0.043135  0.043110  0.043227   
4  0.063937  0.064018  0.064006  0.063894  0.063917  0.063883  0.064030   

      ans_7     ans_8     ans_9  label  
0  0.046800  0.046919  0.046856      0  
1  0.596275  0.596188  0.595964      1  
2  0.654814  0.654682  0.654480      1  
3  0.043106  0.043219  0.043160      0  
4  0.063885  0.064027  0.063945      0  


In [ ]:
sub_lr_b=pd.DataFrame()
sub_lr_b['label']=sub_lr['label'][testa.shape[0]:]

print(sub_lr_b.label.value_counts())
print(sub_lr_b.shape)
print(sub_lr_b.head())

In [40]:
# print( ? /50000)

0.33604


In [41]:
# sub_lr.to_csv('../data/stack/sub_lr.csv')

In [74]:
# sub_lr_b['label'].to_csv('../data/BaselineV1_9_stack.csv',index=False,header=False)

# ==========加强版ensemble=========

In [42]:
# 1.为sub_train,sub_test建立一个mean_label
# 2.利用上面的lr clf进行train_test_split训练，得到lr2_label
# 3.将lr_label与sub_lr_label取均值，然后>0.37处理后递交

In [43]:
# def get_mean(df,N):
#     df['mean_pred']=0
#     for i in range(N):
#         df['mean_pred']+=df['ans_%s'%str(i)]
#     df['mean_pred']=df['mean_pred']/N
#     return df

# str_train=get_mean(sub_train,N=10)
# str_test=get_mean(sub_test,N=10)

# print(str_train.columns,str_test.columns)
# print(str_train.shape,str_test.shape)

# sy=str_train.pop('org_label').values

# sx=str_train['mean_pred'].values.reshape(-1,1)
# stx=str_test['mean_pred'].values.reshape(-1,1)

# fxtrain,fxtest,fytrain,fytest=train_test_split(sx,sy,test_size=0.2,random_state=1)

# lr2=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.08, random_state=1,
#                             solver='liblinear', max_iter=1000)

# lr2.fit(fxtrain,fytrain)
# ftr=lr2.predict_proba(fxtrain)[:,1]
# fpred=lr2.predict_proba(fxtest)[:,1]
# print(f1_score(fytrain,np.where(ftr>0.60,1,0)))
# print(f1_score(fytest,np.where(fpred>0.60,1,0)))

# final=lr2.predict_proba(stx)[:,1]
# sub_lr['f_label']=final

# sub_lr['final']=(sub_lr['f_label']+sub_lr['label'])/2
# sub_lr['final']=sub_lr['final'].apply(lambda x:np.where(x>0.37,1,0))
# print(sub_lr['final'].value_counts())